In [1]:
import os
import openai
from llama_index import SimpleDirectoryReader
from llama_index.embeddings import HuggingFaceEmbedding

from dotenv import load_dotenv



In [2]:
import sys
sys.path.append("..")
from utils_Metrics import get_prebuilt_trulens_recorder

from utils_Adv_RAG_Retriver import build_sentence_window_index

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [3]:
# loading the openai key
load_dotenv()
open_api_key=os.environ['OPENAI_API_KEY']
openai.api_key=open_api_key

In [4]:
# Loading and cchunking the documents

documents=SimpleDirectoryReader(
    input_files=["../data/Machine Learning Engineering with Python-2023.pdf"]
).load_data()


In [5]:
#Checking the summary of chunked documents
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[2].text)
print(len(documents[2].text))
print(len(documents[10].text))
print(len(documents[22].text))




<class 'list'> 

463 

<class 'llama_index.schema.Document'>
Machine Learning Engineering with Python
Second Edition
Copyright © 2023 Packt Publishing
All rights reserved. No part of this book may be reproduced, stored in a retrieval system, or transmitted in 
any form or by any means, without the prior written permission of the publisher, except in the case of brief 
quotations embedded in critical articles or reviews.
Every effort has been made in the preparation of this book to ensure the accuracy of the information 
presented. However, the information contained in this book is sold without warranty, either express or 
implied. Neither the author nor Packt Publishing or its dealers and distributors, will be held liable for any 
damages caused or alleged to have been caused directly or indirectly by this book.
Packt Publishing has endeavored to provide trademark information about all of the companies and products 
mentioned in this book by the appropriate use of capitals. However, Pa

## Basic RAG pipeline

In [6]:
# Combining the all chunked text into single documents
from llama_index import Document

documnet=Document(tetx="\n\n".join([doc.text for doc in documents]))
documnet

Document(id_='de50a7cc-9cf8-49bc-a4a6-e5a6aa5d94b5', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='44136fa355b3678a1146ad16f7e8649e94fb4fc21fe77e8310c060f61caaff8a', text='', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

In [7]:
# Creating index
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

#config llm
llm= OpenAI(model="gpt-3.5-turbo",temperature=0.1) 
#config embedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

service_context=ServiceContext.from_defaults(
    llm=llm,
    embed_model= embed_model
)

index= VectorStoreIndex.from_documents([documnet],
                                       service_context=service_context)

In [8]:
query_engine= index.as_query_engine()

In [9]:
response=query_engine.query("How can we analyzing your own code for security issues")
print(str(response))

One way to analyze your own code for security issues is to conduct a thorough code review. This involves carefully examining the code line by line to identify any potential vulnerabilities or weaknesses. Additionally, using automated code analysis tools can help to identify common security issues such as SQL injection or cross-site scripting. Regularly updating and patching your code, as well as following secure coding practices, can also help to mitigate security risks.


## Evaluation setup using TruLens

In [10]:
eval_questions = []
with open('../eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

How can we analyzing your own code for security issues


In [11]:
# Adding new question:
new_question = "How to publish in ECR"
eval_questions.append(new_question)


In [12]:
print(eval_questions)

['How can we analyzing your own code for security issues', 'How to publish in ECR']


In [13]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

In [15]:
import sys
sys.path.append("..")
from utils_Metrics import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")


✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [16]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

In [17]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [18]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Context Relevance,Answer Relevance,Context Relevance_calls,Answer Relevance_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_fc20ff168bd57405fe069d80deb342cc,"""How can we analyzing your own code for securi...","""One way to analyze your own code for security...",-,"{""record_id"": ""record_hash_fc20ff168bd57405fe0...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-03T13:08:50.125238"", ""...",2023-12-03T13:08:57.999792,0.0,0.9,[{'args': {'question': 'How can we analyzing y...,[{'args': {'prompt': 'How can we analyzing you...,7,198,0.000335
1,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_e26f2a5947684d8212f903f8ef261f74,"""How to publish in ECR""","""To publish in ECR, you can follow these steps...",-,"{""record_id"": ""record_hash_e26f2a5947684d8212f...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-03T13:08:58.232392"", ""...",2023-12-03T13:09:10.466284,NaN,NaN,NaN,NaN,12,247,0.000436


## Advanced RAG pipeline

### 1. Sentence Window retrieval

In [19]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [20]:

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")


sentence_index = build_sentence_window_index(
    documnet,
    llm,
    embed_model=embed_model,
    save_dir="sentence_index"
)

In [21]:
from utils_Adv_RAG_Retriver import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [22]:
window_response = sentence_window_engine.query(
    "How to publish in ECR"
)
print(str(window_response))

To publish a container in ECR, you need to follow these steps:

1. Create an ECR repository using the AWS CLI command `aws ecr create-repository`. Specify the repository name and the desired region. Make sure to enable image scanning by setting `scanOnPush=true`.

2. After creating the repository, you will receive metadata that includes the repository URI. Take note of this URI for later steps.

3. Log in to the container registry using the `docker login` command in the Terminal. Use the repository URI provided in the metadata.

4. Build your container image using a Dockerfile. In the Dockerfile, specify the base image, copy the necessary files, and define any other required configurations.

5. Tag your container image using the repository URI and a desired tag.

6. Push the tagged container image to the ECR repository using the `docker push` command.

By following these steps, you can successfully publish your container in ECR.


In [23]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = "Sentence Window Query Engine"
)

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [24]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

How can we analyzing your own code for security issues
To analyze your own code for security issues, you can use the open-source Bandit package. Bandit is a linter that is specifically designed to find security issues in source code. It allows you to perform static analysis of your code and check for vulnerabilities that may have been introduced during development. Bandit is easy to run and can help you identify potential security risks in your code.
How to publish in ECR
To publish a container in ECR, you need to follow these steps:

1. Create an ECR repository using the AWS CLI command `aws ecr create-repository`. Specify the repository name and the desired region. Make sure to enable image scanning by setting the `--image-scanning-configuration` flag to `scanOnPush=true`.

2. After creating the repository, you will receive metadata that includes the repository URI. Take note of this URI for later steps.

3. Log in to the container registry using the `docker login` command in the Ter

In [25]:
tru.get_leaderboard(app_ids=[])

,Context Relevance,Answer Relevance,Groundedness,latency,total_cost
app_id,,,,,
Sentence Window Query Engine,0.6,0.95,0.75,16.0,0.001121


In [26]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [27]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,Sentence Window Query Engine,"{""app_id"": ""Sentence Window Query Engine"", ""ta...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_e72f098031eba897268f3fdbf3d3dd51,"""How can we analyzing your own code for securi...","""To analyze your own code for security issues,...",-,"{""record_id"": ""record_hash_e72f098031eba897268...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-03T13:10:35.022155"", ""...",2023-12-03T13:10:46.619578,0.9,0.8,0.75,[{'args': {'prompt': 'How can we analyzing you...,[{'args': {'question': 'How can we analyzing y...,[{'args': {'source': 'Since machine learning s...,11,681,0.001060
1,Sentence Window Query Engine,"{""app_id"": ""Sentence Window Query Engine"", ""ta...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_1d7830f2ad176bce6546d2a150c5bdaa,"""How to publish in ECR""","""To publish a container in ECR, you need to fo...",-,"{""record_id"": ""record_hash_1d7830f2ad176bce654...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-03T13:10:46.942843"", ""...",2023-12-03T13:11:08.009491,1.0,0.4,0.25,"[{'args': {'prompt': 'How to publish in ECR', ...",[{'args': {'question': 'How to publish in ECR'...,[{'args': {'source': 'ENV PATH=/root/.local:$P...,21,724,0.001182
